In [3]:
from torch.utils.data import Dataset, DataLoader
import torch
from transformers import AutoTokenizer
import torch.nn as nn
import math

# TOKENİZASYON

In [4]:
# =====================================
# 1️⃣ Seq2Seq Tokenization Dataset (T5/BART uyumlu)
# =====================================
class Seq2SeqDataset(Dataset):
    """
    Tokenization pipeline for seq2seq LLMs (T5, BART, mBART)
    Includes encoder/decoder tokenization, attention masks, shifted decoder input.
    """
    def __init__(self, sources, targets, tokenizer_name="t5-small", max_length=64):
        self.sources = sources
        self.targets = targets
        self.tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
        self.max_length = max_length

        # Seq2Seq modellerinde decoder_start_token_id önemli
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token  # zorunlu pad token
        self.decoder_start_token_id = self.tokenizer.pad_token_id if self.tokenizer.bos_token_id is None else self.tokenizer.bos_token_id

    def __len__(self):
        return len(self.sources)

    def __getitem__(self, idx):
        # -------------------
        # Encoder tokenization
        # -------------------
        enc = self.tokenizer(
            self.sources[idx],
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        # -------------------
        # Decoder tokenization
        # -------------------
        dec = self.tokenizer(
            self.targets[idx],
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        # -------------------
        # Decoder input (shifted right)
        # -------------------
        decoder_input_ids = torch.cat(
            [torch.full((1,1), self.decoder_start_token_id), dec['input_ids'][:, :-1]], dim=1
        )

        return {
            'encoder_input_ids': enc['input_ids'].squeeze(0),
            'encoder_attention_mask': enc['attention_mask'].squeeze(0),
            'decoder_input_ids': decoder_input_ids.squeeze(0),
            'decoder_attention_mask': dec['attention_mask'].squeeze(0),
            'decoder_target_ids': dec['input_ids'].squeeze(0)
        }

# =====================================
# 2️⃣ Collate function
# =====================================
def collate_fn(batch):
    enc_ids = torch.stack([item['encoder_input_ids'] for item in batch])
    enc_mask = torch.stack([item['encoder_attention_mask'] for item in batch])
    dec_ids = torch.stack([item['decoder_input_ids'] for item in batch])
    dec_mask = torch.stack([item['decoder_attention_mask'] for item in batch])
    dec_target = torch.stack([item['decoder_target_ids'] for item in batch])
    return {
        'encoder_input_ids': enc_ids,
        'encoder_attention_mask': enc_mask,
        'decoder_input_ids': dec_ids,
        'decoder_attention_mask': dec_mask,
        'decoder_target_ids': dec_target
    }

# =====================================
# 3️⃣ Örnek kullanım
# =====================================
input_texts = [
    "Merhaba dünya!",
    "Transformers çok güçlü.",
    "Memory-efficient pipeline."
]

target_texts = [
    "Hello world!",
    "Transformers are powerful.",
    "Super useful pipeline."
]

dataset = Seq2SeqDataset(input_texts, target_texts, tokenizer_name="t5-small", max_length=16)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True, collate_fn=collate_fn)

# =====================================
# 4️⃣ Test bir batch
# =====================================
device = "cuda" if torch.cuda.is_available() else "cpu"

for batch in dataloader:
    batch = {k:v.to(device) for k,v in batch.items()}
    print("Encoder Input IDs:", batch['encoder_input_ids'].shape)
    print("Encoder Attention Mask:", batch['encoder_attention_mask'].shape)
    print("Decoder Input IDs:", batch['decoder_input_ids'].shape)
    print("Decoder Attention Mask:", batch['decoder_attention_mask'].shape)
    print("Decoder Target IDs:", batch['decoder_target_ids'].shape)
    break


Encoder Input IDs: torch.Size([2, 16])
Encoder Attention Mask: torch.Size([2, 16])
Decoder Input IDs: torch.Size([2, 16])
Decoder Attention Mask: torch.Size([2, 16])
Decoder Target IDs: torch.Size([2, 16])


# DECODER + ENCODER Ortak Class'lar


In [ ]:
class DropPath(nn.Module):
    def __init__(self, drop_prob = 0.0):
        super().__init__()
        self.drop_prob = drop_prob
    
    def forward(self,x):
        if self.drop_prob == 0.0 or not self.training:
            return x

        keep_prob = 1 - self.drop_prob
        shape = (x.size(0) , ) + (1,) * (x.ndim - 1)
        random_tensor = keep_prob + torch.rand(shape , dtype=x.dtype , device = x.device)
        random_tensor.floor_()
        return x.div(keep_prob) *  random_tensor

In [ ]:
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size, embed_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
    def forward(self, x):
        return self.embedding(x)

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, embed_dim, max_len=5000):
        super().__init__()
        pe = torch.zeros(max_len, embed_dim)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, embed_dim, 2).float() * (-math.log(10000.0) / embed_dim))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        seq_len = x.size(1)
        return x + self.pe[:, :seq_len, :]

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, embed_dim, num_heads=16, dp=0.1):
        super().__init__()
        assert embed_dim % num_heads == 0
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads
        self.scale = self.head_dim ** -0.5

        self.q_proj = nn.Linear(embed_dim, embed_dim)
        self.k_proj = nn.Linear(embed_dim, embed_dim)
        self.v_proj = nn.Linear(embed_dim, embed_dim)
        self.out_proj = nn.Linear(embed_dim, embed_dim)
        self.dropout = nn.Dropout(dp)

    def split_heads(self, x):
        B, T, C = x.size()
        return x.view(B, T, self.num_heads, self.head_dim).transpose(1, 2)

    def combine_heads(self, x):
        B, _, T, _ = x.size()
        return x.transpose(1, 2).contiguous().view(B, T, self.num_heads * self.head_dim)

    def forward(self, query, key, value, mask=None):
        Q = self.split_heads(self.q_proj(query))
        K = self.split_heads(self.k_proj(key))
        V = self.split_heads(self.v_proj(value))
        scores = torch.matmul(Q, K.transpose(-2, -1)) * self.scale
        if mask is not None:
            scores = scores.masked_fill(mask == 0, float('-inf'))
        attn = torch.softmax(scores, dim=-1)
        attn = self.dropout(attn)
        out = torch.matmul(attn, V)
        return self.out_proj(self.combine_heads(out))

In [ ]:
class FeedForward(nn.Module):
    def __init__(self, embed_dim, expansion=8, dp=0.1, use_swiglu=False):
        super().__init__()
        if use_swiglu:
            # SwiGLU activation
            self.net = nn.Sequential(
                nn.Linear(embed_dim, embed_dim * expansion * 2),
                nn.SiLU(),
                nn.Dropout(dp),
                nn.Linear(embed_dim * expansion, embed_dim),
                nn.Dropout(dp)
            )
        else:
            self.net = nn.Sequential(
                nn.Linear(embed_dim, embed_dim * expansion),
                nn.GELU(),
                nn.Dropout(dp),
                nn.Linear(embed_dim * expansion, embed_dim),
                nn.Dropout(dp)
            )
    def forward(self, x):
        return self.net(x)

# ENCODER

In [ ]:
class TransformerEncoderBlockLLM(nn.Module):
    def __init__(self, embed_dim=1024, num_heads=16, dp=0.1, drop_path=0.1, expansion=8, use_swiglu=False):
        super().__init__()
        self.norm1 = nn.LayerNorm(embed_dim)
        self.norm2 = nn.LayerNorm(embed_dim)

        self.self_attn = MultiHeadAttention(embed_dim, num_heads, dp)
        self.ffn = FeedForward(embed_dim, expansion, dp, use_swiglu)

        self.drop_path = DropPath(drop_path)
        self.gamma_1 = nn.Parameter(torch.ones(embed_dim) * 1e-2)
        self.gamma_2 = nn.Parameter(torch.ones(embed_dim) * 1e-2)

    def forward(self, x, mask=None):
        # Self-Attention
        attn_out = self.self_attn(self.norm1(x), self.norm1(x), self.norm1(x), mask)
        x = x + self.drop_path(self.gamma_1 * attn_out)
        # FeedForward
        ffn_out = self.ffn(self.norm2(x))
        x = x + self.drop_path(self.gamma_2 * ffn_out)
        return x

In [14]:
class TransformersEncoderLLM(nn.Module):
    def __init__(self, vocab_size , embed_dim = 1024 , num_layers = 12 , dp = 0.1 ,num_heads=16 ,  expansion = 8 , max_len= 5000 , drop_path = 0.1 , use_swiglu =False):
        super().__init__()

        self.tok_emb = TokenEmbedding(vocab_size,embed_dim)
        self.pos_enc = PositionalEncoding(embed_dim , max_len)
        self.layers = nn.ModuleList(
            [TransformerEncoderBlockLLM(embed_dim , num_heads , dp , drop_path , expansion , use_swiglu) for _ in range(num_layers)]
            )
        self.norm = nn.LayerNorm(embed_dim)
    
    def forward(self,src_tokens , src_mask =None):
        x = self.tok_emb(src_tokens)
        x = self.pos_enc(x)
        for layer in self.layers:
            x = layer(x,mask = src_mask)
        x = self.norm(x)
        return x

# DECODER

In [ ]:
class TransformerDecoderBlockLLM(nn.Module):
    def __init__(self, embed_dim=1024, num_heads=16, dp=0.1, drop_path=0.1, expansion=8, use_swiglu=False):
        super().__init__()
        self.norm1 = nn.LayerNorm(embed_dim)
        self.norm2 = nn.LayerNorm(embed_dim)
        self.norm3 = nn.LayerNorm(embed_dim)

        self.self_attn = MultiHeadAttention(embed_dim, num_heads, dp)
        self.cross_attn = MultiHeadAttention(embed_dim, num_heads, dp)
        self.ffn = FeedForward(embed_dim, expansion, dp, use_swiglu)

        self.drop_path = DropPath(drop_path)
        self.gamma_1 = nn.Parameter(torch.ones(embed_dim) * 1e-2)
        self.gamma_2 = nn.Parameter(torch.ones(embed_dim) * 1e-2)
        self.gamma_3 = nn.Parameter(torch.ones(embed_dim) * 1e-2)

    def forward(self, x, enc_out=None, self_mask=None, enc_mask=None):
        # Masked Self-Attention
        attn_out = self.self_attn(self.norm1(x), self.norm1(x), self.norm1(x), self_mask)
        x = x + self.drop_path(self.gamma_1 * attn_out)

        # Cross-Attention
        if enc_out is not None:
            cross_out = self.cross_attn(self.norm2(x), self.norm2(enc_out), self.norm2(enc_out), enc_mask)
            x = x + self.drop_path(self.gamma_2 * cross_out)

        # FeedForward
        ffn_out = self.ffn(self.norm3(x))
        x = x + self.drop_path(self.gamma_3 * ffn_out)

        return x

In [21]:
class TransformerDecoderLLM(nn.Module):
    def __init__(self, vocab_size, embed_dim=1024, num_layers=12, num_heads=16, dp=0.1, drop_path=0.1, expansion=8, max_len=5000, use_swiglu=False):
        super().__init__()
        self.embedding = TokenEmbedding(vocab_size, embed_dim)
        self.pos_encoding = PositionalEncoding(embed_dim, max_len)
        self.layers = nn.ModuleList([
            TransformerDecoderBlockLLM(embed_dim, num_heads, dp, drop_path, expansion, use_swiglu) for _ in range(num_layers)
        ])
        self.norm = nn.LayerNorm(embed_dim)
        self.lm_head = nn.Linear(embed_dim, vocab_size)

    def forward(self, x, enc_out=None, self_mask=None, enc_mask=None):
        x = self.embedding(x)
        x = self.pos_encoding(x)
        for layer in self.layers:
            x = layer(x, enc_out, self_mask, enc_mask)
        x = self.norm(x)
        logits = self.lm_head(x)
        return logits

----
# Yukarıda bulunan kodlarla alakalı ; 
---

## Soru 1-) DropPath classındaki parametreler ne işe yarıyor? İşleyişi nasıl oluyor?Adım adım açıkayalım.

```python
class DropPath(nn.Module):
    def __init__(self, drop_prob = 0.0):
        super().__init__()
        self.drop_prob = drop_prob
    
    def forward(self,x):
        if self.drop_prob == 0.0 or not self.training:
            return x

        keep_prob = 1 - self.drop_prob
        shape = (x.size(0) , ) + (1,) * (x.ndim - 1)
        random_tensor = keep_prob + torch.rand(shape , dtype=x.dtype , device = x.device)
        random_tensor.floor_()
        return x.div(keep_prob) *  random_tensor

### 🎯 1️⃣ Amaç — DropPath ne işe yarar?

* Normal Dropout, tensor içindeki bazı nöron aktivasyonlarını 0 yapar.
* DropPath ise, bütün bir residual dalını (katmanı) tamamen atlayabilir.
*  Bu sayede modelin derin katmanları rastgele “devre dışı” bırakılır,
*  Böylece model, daha sığ yolları da öğrenmeye zorlanır (regularization).

### 🧩 2️⃣ Parametreler

| Parametre   | Anlamı            | Açıklama                                                                    |
| ----------- | ----------------- | --------------------------------------------------------------------------- |
| `drop_prob` | float (0–1 arası) | Katmanı atlama olasılığı. Örneğin 0.1 → %10 ihtimalle residual dal atlanır. |
| `x`         | Tensor            | Modelden geçen aktivasyon (örneğin `F(x)` sonucu).                          |


## ⚙️ 3️⃣ Adım Adım İşleyiş


### 🔸 Adım 1: 
```python
if self.drop_prob == 0.0 or not self.training:
    return x


* DropPath sadece eğitim sırasında aktiftir, inference (tahmin) sırasında devre dışı kalır.

* Eğer drop_prob = 0 ise, hiçbir katman atlanmaz (normal çalışır).

### 🔸 Adım 2:
```python
keep_prob = 1 - self.drop_prob
```


* “Katmanı koruma olasılığı” belirlenir.
* Örnek: drop_prob = 0.1 → keep_prob = 0.9

### 🔸 Adım 3:
```python
shape = (x.size(0),) + (1,) * (x.ndim - 1)
```


* Her batch örneği için tek bir rastgele sayı üretmek üzere bir şekil oluşturulur.

Örnek:

> x boyutu: (B, L, D)

> shape: (B, 1, 1)


* 👉 Bu sayede her örnek için tüm zaman adımlarına aynı maske uygulanır
(yani “katman bazında dropout” yapılır)

### 🔸 Adım 4:
```python
random_tensor = keep_prob + torch.rand(shape, dtype=x.dtype, device=x.device)
```


* torch.rand(shape) → 0 ile 1 arasında rastgele değerler üretir.

* Buna keep_prob eklenir → dağılım sağa kayar.
* Örnek: keep_prob = 0.9 → random_tensor değerleri 0.9–1.9 aralığında olur.

### 🔸 Adım 5:
```python
random_tensor.floor_()
```


floor_() işlemi ile:

* Değer < 1.0 → 0 olur (katman atlandı)

* Değer ≥ 1.0 → 1 olur (katman korundu)

## 🧠  Özet

| Aşama                  | İşlev                                  |
| ---------------------- | -------------------------------------- |
| `drop_prob`            | Katmanı atlama oranı belirler          |
| `keep_prob`            | Katmanı koruma olasılığı               |
| `random_tensor`        | Hangi örneklerin atlanacağını belirler |
| `x.div(keep_prob)`     | Aktivasyon ölçeğini dengeler           |
| `* random_tensor`      | Katmanı aktif veya pasif hale getirir  |
| `if not self.training` | Sadece eğitim sırasında etkin          |


### 📊  Görsel Örnek (basitleştirilmiş)
```bash
Input (batch_size=3)
x = [[1,1,1], [2,2,2], [3,3,3]]

drop_prob=0.33 → keep_prob=0.67
random_tensor = [1, 0, 1]
output = [[1.5,1.5,1.5], [0,0,0], [4.5,4.5,4.5]]

----
## Soru 2-) PositionelEncoding işlemindeki parametreler nedir? Torch içerisinde çağırılan .exp ve .arange gibi terimler ne işe yarar? Adım adım açıklayınız.
```python

class PositionalEncoding(nn.Module):
    def __init__(self, embed_dim, max_len=5000):
        super().__init__()
        pe = torch.zeros(max_len, embed_dim)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, embed_dim, 2).float() * (-math.log(10000.0) / embed_dim))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        seq_len = x.size(1)
        return x + self.pe[:, :seq_len, :]

### ⚙️ 2️⃣ PositionalEncoding Sınıfı — Adım Adım Açıklama

* Transformerlarda “pozisyon bilgisi”, sıradaki kelimelerin **konum farklarını** modele öğretmek için kullanılır.  
* Bu sınıf, **her pozisyona sinüs ve kosinüs temelli benzersiz bir vektör** ekler.


### 🔹 Adım 1: 
```python
pe = torch.zeros(max_len, embed_dim)


* pe: Pozisyonel kodlamaları tutacak bir matris.

* Boyut: (max_len, embed_dim)

* max_len: Cümlenin maksimum uzunluğu (ör. 5000 token).

* embed_dim: Her kelimenin embedding boyutu (ör. 512).

* Başlangıçta sıfırlarla doldurulur.

### 🔹 Adım 2:
```python
position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
```


* torch.arange(0, max_len) → 0’dan max_len-1’e kadar bir sayı dizisi üretir.
(örneğin [0, 1, 2, 3, ... 4999])

* .unsqueeze(1) → 1 boyut ekler → (max_len, 1) haline getirir.

- Bu vektör her pozisyonun indeksini temsil eder.

### 🔹 Adım 3:
```python
div_term = torch.exp(torch.arange(0, embed_dim, 2).float() * (-math.log(10000.0) / embed_dim))
```

Bu kısım frekans ölçeklerini hesaplar:

* torch.arange(0, embed_dim, 2) → 0, 2, 4, ... (her çift indeks için)

* (-math.log(10000.0) / embed_dim) → sabit bir oran belirler.

* torch.exp(...) → bu oranı üstel (exponential) fonksiyonla ölçeklendirir.

👉 Amaç: Her boyutta farklı bir “dalga frekansı” oluşturmak.
* Yani küçük boyutlar yavaş değişen sinüs/kosinüs, büyük boyutlar hızlı değişen frekans üretir.

### 🔹 Adım 4:
```python
pe[:, 0::2] = torch.sin(position * div_term)
pe[:, 1::2] = torch.cos(position * div_term)
```



* Çift indeksler (0,2,4,...) için sinüs,

* Tek indeksler (1,3,5,...) için kosinüs uygulanır.

🔸 Sonuç:
* Her pozisyon için embed_dim uzunluğunda benzersiz bir vektör elde edilir.
* Bu vektörlerin sinüs ve kosinüs kombinasyonu, mutlak pozisyonu ve göreli farkları kodlar.

### 🔹 Adım 5:
```python
pe = pe.unsqueeze(0)
self.register_buffer('pe', pe)
```

* .unsqueeze(0) → batch boyutu ekler: (1, max_len, embed_dim)

* register_buffer → pe tensörünü modelin parametresi yapmaz, ama state_dict’e dahil eder.
(yani requires_grad=False, ama modelle birlikte GPU’ya taşınır.)

### 🔹 Adım 6:
```python
def forward(self, x):
    seq_len = x.size(1)
    return x + self.pe[:, :seq_len, :]
```


* Giriş x boyutu: (batch, seq_len, embed_dim)

* self.pe[:, :seq_len, :] → sadece ilgili uzunluk kadar pozisyonel vektör alır.

* x + self.pe → embedding’e pozisyon bilgisi eklenir.

## 🧠 Ek Bilgi — torch Fonksiyonlarının Görevleri

| Fonksiyon                  | Açıklama                                            |
| -------------------------- | --------------------------------------------------- |
| `torch.arange(start, end)` | Belirtilen aralıkta ardışık sayılar üretir.         |
| `.unsqueeze(dim)`          | Belirtilen boyuta 1 ekler (ör. (5000,) → (5000,1)). |
| `torch.exp(tensor)`        | Her elemanın e^x’ini (üstelini) alır.               |
| `torch.sin`, `torch.cos`   | Trigonometrik sinüs ve kosinüs fonksiyonlarıdır.    |


### 🔸 Özet:

Bu yapı sayesinde model, sıradaki token’ların:

* sıralarını,

* göreli uzaklıklarını,

* ve bağlamsal ilişkilerini

öğrenebilir hale gelir.

----

## Soru 3-) FeedForward ne işe yarar ? Silu ve Gelu fonksiyonlarının farkı nedir? Neden böyle bir ayrışım yapıyoruz ? 
```python 
class FeedForward(nn.Module):
    def __init__(self, embed_dim, expansion=8, dp=0.1, use_swiglu=False):
        super().__init__()
        if use_swiglu:
            # SwiGLU activation
            self.net = nn.Sequential(
                nn.Linear(embed_dim, embed_dim * expansion * 2),
                nn.SiLU(),
                nn.Dropout(dp),
                nn.Linear(embed_dim * expansion, embed_dim),
                nn.Dropout(dp)
            )
        else:
            self.net = nn.Sequential(
                nn.Linear(embed_dim, embed_dim * expansion),
                nn.GELU(),
                nn.Dropout(dp),
                nn.Linear(embed_dim * expansion, embed_dim),
                nn.Dropout(dp)
            )
    def forward(self, x):
        return self.net(x)

### 🔹 1️⃣ FeedForward (FFN) Katmanı Nedir?

* Transformer’ın her encoder veya decoder bloğunun içinde, attention katmanından sonra yer alan bir tam bağlantılı (fully-connected) yapıdır.

Amaç:

➡️ Her token’ın (kelimenin) temsilini bağımsız olarak dönüştürmek,

➡️ Modele nonlineerlik ve daha yüksek temsil gücü kazandırmak.

### 🧠 Genel İşleyiş

Girdi:
* Attention katmanından gelen x (şekil: [batch_size, seq_len, embed_dim])

İlk Linear:
* Her token vektörünü embed_dim’den embed_dim * expansion boyutuna çıkarır.
(Bu genişletme, bilgiyi daha zengin bir uzayda işlemeye yarar.)

Aktivasyon Fonksiyonu (GELU veya SiLU):
* Nonlineer dönüşüm sağlar — modelin karmaşık ilişkileri öğrenmesine yardımcı olur.

Dropout:
* Regularization, yani overfitting’i önleme.

İkinci Linear:
* Boyutu tekrar embed_dim’e indirir (geri sıkıştırır).

### 🔹 2️⃣ SiLU ve GELU Farkı

| Özellik               | **SiLU (Sigmoid Linear Unit)**                           | **GELU (Gaussian Error Linear Unit)**                            |
| --------------------- | -------------------------------------------------------- | ---------------------------------------------------------------- |
| **Formül**            | `x * sigmoid(x)`                                         | `x * Φ(x)` <br> (Φ: normal dağılımın kümülatif fonksiyonu)       |
| **Yaklaşım**          | Sigmoid’e dayalı, yumuşak ReLU benzeri                   | Gaussian olasılık temelli                                        |
| **Davranış**          | Küçük pozitif girişlerde yumuşak, büyük pozitiflerde ≈ x | Benzer şekilde yumuşak ama Gaussian eğrisiyle ölçeklenir         |
| **Grafiksel Görünüm** | Daha düz geçişli, stabil                                 | Hafif asimetrik, “daha doğal” aktivasyon sağlar                  |
| **Kullanım**          | Genelde **SwiGLU** gibi kapı mekanizmalarında            | Transformer modellerinde (örneğin BERT, GPT, ViT) sık kullanılır |
| **Avantaj**           | Hesaplaması daha ucuz ve kararlı                         | Empirik olarak bazı modellerde daha yüksek doğruluk sağlar       |


### 🔹 3️⃣ Neden “SwiGLU” Ayrımı Yapıyoruz?

Kodda bir parametre var:

```python
use_swiglu=True
```


* Bu, FeedForward içinde farklı aktivasyon kombinasyonlarını denemeye yarıyor.

### 🧩 SwiGLU Nedir?

* SwiGLU (Swish-Gated Linear Unit), SiLU temelli bir kapılı (gated) yapı.
Burada Linear katmandan iki ayrı parça çıkarılır:

* biri gate (kapı),

* diğeri değer (value) kısmı.

Yani:

* y = (x * W1) * SiLU(x * W2)

* Bu yapı, modelin bilgiyi seçici olarak geçirmesine olanak tanır —
tıpkı attention’daki “hangi bilgi önemli?” mantığı gibi.

### Avantaj:

* Daha iyi bilgi akışı,

* Daha az “ölü nöron”,

* Düşük hesaplama maliyetiyle yüksek performans.

### 🔹 4️⃣ Özetle Adım Adım İşleyiş

| Adım | İşlem                                         | Açıklama                              |
| ---- | --------------------------------------------- | ------------------------------------- |
| 1    | `nn.Linear(embed_dim, embed_dim * expansion)` | Boyutu genişletir (bilgi uzayı büyür) |
| 2    | `nn.GELU()` veya `nn.SiLU()`                  | Nonlineer dönüşüm                     |
| 3    | `nn.Dropout(dp)`                              | Overfitting önleme                    |
| 4    | `nn.Linear(embed_dim * expansion, embed_dim)` | Boyutu geri indirir                   |
| 5    | `nn.Dropout(dp)`                              | Son düzenleme, regularization         |


### 🔹 5️⃣ Kısa Sonuç

| Parametre    | Anlamı                                                             |
| ------------ | ------------------------------------------------------------------ |
| `embed_dim`  | Girdi vektör boyutu                                                |
| `expansion`  | Katman genişleme oranı (genelde 4–8 arası)                         |
| `dp`         | Dropout oranı                                                      |
| `use_swiglu` | True ise SiLU tabanlı SwiGLU kullanılır, False ise GELU klasik FFN |


```bash
 ┌────────────────────────────┐
 │   Girdi: x (batch, seq, embed_dim) │
 └──────────────┬─────────────┘
                │
                ▼
      ┌────────────────────┐
      │ Linear (embed_dim → embed_dim * expansion) │
      └────────────────────┘
                │
                ▼
          ┌────────────┐
          │  GELU      │   ← Nonlineer dönüşüm (Gaussian temelli)
          └────────────┘
                │
                ▼
          ┌────────────┐
          │ Dropout(dp)│
          └────────────┘
                │
                ▼
      ┌────────────────────┐
      │ Linear (embed_dim * expansion → embed_dim) │
      └────────────────────┘
                │
                ▼
          ┌────────────┐
          │ Dropout(dp)│
          └────────────┘
                │
                ▼
 ┌────────────────────────────┐
 │  Çıktı: y (aynı boyut: embed_dim) │
 └────────────────────────────┘


---

## Soru 4-) TransformersEncoderBlockLLM sınıfı ne işe yarıyor.Gamma parametreleri nedir ? Çarpım için kullanılan 1e-2 gibi değerler azalsa ya da artsa nasıl bir değişim beklenir? Ayrıca bu classın forward fonksiyonunda ne gibi işlemler yapılıyor ? 
```python
class TransformerEncoderBlockLLM(nn.Module):
    def __init__(self, embed_dim=1024, num_heads=16, dp=0.1, drop_path=0.1, expansion=8, use_swiglu=False):
        super().__init__()
        self.norm1 = nn.LayerNorm(embed_dim)
        self.norm2 = nn.LayerNorm(embed_dim)

        self.self_attn = MultiHeadAttention(embed_dim, num_heads, dp)
        self.ffn = FeedForward(embed_dim, expansion, dp, use_swiglu)

        self.drop_path = DropPath(drop_path)
        self.gamma_1 = nn.Parameter(torch.ones(embed_dim) * 1e-2)
        self.gamma_2 = nn.Parameter(torch.ones(embed_dim) * 1e-2)

    def forward(self, x, mask=None):
        # Self-Attention
        attn_out = self.self_attn(self.norm1(x), self.norm1(x), self.norm1(x), mask)
        x = x + self.drop_path(self.gamma_1 * attn_out)
        # FeedForward
        ffn_out = self.ffn(self.norm2(x))
        x = x + self.drop_path(self.gamma_2 * ffn_out)
        return x

### ⚙️ Soru 4 - TransformerEncoderBlockLLM
#### 1️⃣ Amaç ve İşlevi

* TransformerEncoderBlockLLM sınıfı, Transformer mimarisindeki tek bir encoder bloğunu temsil eder.

Bir bloğun temel işlemleri şunlardır:

* Self-Attention: Girdi sırasındaki her token’ın diğer token’larla ilişkisini öğrenir.

* FeedForward (FFN): Token başına öğrenilen özellikleri dönüştürür ve model kapasitesini artırır.

- LayerNorm ve DropPath: Eğitimi stabilize eder ve düzenliliği artırır.

Kısaca:

* “Girdi → Normalizasyon → Self-Attention → Skip Connection → FFN → Skip Connection → Çıkış”

#### 2️⃣ gamma_1 ve gamma_2 Parametreleri
```python
self.gamma_1 = nn.Parameter(torch.ones(embed_dim) * 1e-2)
self.gamma_2 = nn.Parameter(torch.ones(embed_dim) * 1e-2)
```



* Bunlar ölçekleme faktörleri olarak çalışır.

* attn_out ve ffn_out değerlerine çarpılır önce DropPath, sonra residual ekleme yapılır:
```python

x = x + self.drop_path(self.gamma_1 * attn_out)
x = x + self.drop_path(self.gamma_2 * ffn_out)
```


Amaç: residual bağlantıdaki katkıyı başlangıçta küçük tutmak, böylece öğrenme daha stabil olur.

* Eğer 1e-2 değeri artarsa: attention/FFN çıktısı daha fazla eklenir → gradient daha hızlı yayılır → risk: instabilite, patlayan değerler.

* Eğer azalırsa: blok çıktısı daha az katkıda bulunur → yavaş öğrenme, yavaş adaptasyon

#### 3️⃣ Forward Fonksiyonu Adım Adım

```python 
def forward(self, x, mask=None):
    # 1️⃣ Normalizasyon
    attn_out = self.self_attn(self.norm1(x), self.norm1(x), self.norm1(x), mask)
    
    # 2️⃣ DropPath + Gamma çarpımı + Residual
    x = x + self.drop_path(self.gamma_1 * attn_out)

    # 3️⃣ FeedForward
    ffn_out = self.ffn(self.norm2(x))
    
    # 4️⃣ DropPath + Gamma çarpımı + Residual
    x = x + self.drop_path(self.gamma_2 * ffn_out)

    return x

| Adım | İşlem                                              | Amaç                                        |
| ---- | -------------------------------------------------- | ------------------------------------------- |
| 1    | `self.norm1(x)`                                    | LayerNorm ile input stabilizasyonu          |
| 2    | `self.self_attn(...)`                              | Token’lar arası ilişkileri öğrenme          |
| 3    | `self.gamma_1 * attn_out`                          | Residual katkısını ölçekleme                |
| 4    | `self.drop_path(...)`                              | Stochastic Depth, blok rastgele atlanabilir |
| 5    | `x + ...`                                          | Residual bağlantı ekleme                    |
| 6    | `self.norm2(x)` → `self.ffn(...)`                  | FeedForward, token başına dönüşüm           |
| 7    | `self.gamma_2 * ffn_out` + `drop_path` + `x + ...` | Residual + Stabilizasyon                    |


#### 4️⃣ Özet

* TransformerEncoderBlockLLM: tek encoder bloğu, Self-Attention + FFN + Residual + LayerNorm + DropPath içerir.

* gamma_1 ve gamma_2: residual katkıyı başlangıçta küçük tutar. Değeri değiştirirsen stabilite ve öğrenme hızı değişir.

```bash4
* forward:

* LayerNorm

* Self-Attention → scaled + drop_path → residual

* FeedForward → scaled + drop_path → residual

* Çıkış

---

## Soru 5-) TransformerDecoderBlockLLM sınıfında bulunan işlem adımları nedir?Oluşturulan gamma parametreleri ne işe yarar? Self Attention ve Cross Attention aynı class içerisinde ise modele self kullanacağını ya da cross attention kullanacağını nasıl aktarıyoruz ? 
```python

class TransformerDecoderBlockLLM(nn.Module):
    def __init__(self, embed_dim=1024, num_heads=16, dp=0.1, drop_path=0.1, expansion=8, use_swiglu=False):
        super().__init__()
        self.norm1 = nn.LayerNorm(embed_dim)
        self.norm2 = nn.LayerNorm(embed_dim)
        self.norm3 = nn.LayerNorm(embed_dim)

        self.self_attn = MultiHeadAttention(embed_dim, num_heads, dp)
        self.cross_attn = MultiHeadAttention(embed_dim, num_heads, dp)
        self.ffn = FeedForward(embed_dim, expansion, dp, use_swiglu)

        self.drop_path = DropPath(drop_path)
        self.gamma_1 = nn.Parameter(torch.ones(embed_dim) * 1e-2)
        self.gamma_2 = nn.Parameter(torch.ones(embed_dim) * 1e-2)
        self.gamma_3 = nn.Parameter(torch.ones(embed_dim) * 1e-2)

    def forward(self, x, enc_out=None, self_mask=None, enc_mask=None):
        # Masked Self-Attention
        attn_out = self.self_attn(self.norm1(x), self.norm1(x), self.norm1(x), self_mask)
        x = x + self.drop_path(self.gamma_1 * attn_out)

        # Cross-Attention
        if enc_out is not None:
            cross_out = self.cross_attn(self.norm2(x), self.norm2(enc_out), self.norm2(enc_out), enc_mask)
            x = x + self.drop_path(self.gamma_2 * cross_out)

        # FeedForward
        ffn_out = self.ffn(self.norm3(x))
        x = x + self.drop_path(self.gamma_3 * ffn_out)

        return x

### ⚙️ Soru 5 - TransformerDecoderBlockLLM
#### 1️⃣ Amaç ve İşlevi

* TransformerDecoderBlockLLM, tek bir decoder bloğunu temsil eder ve üç ana bileşenden oluşur:

* Masked Self-Attention → Decoder kendi önceki token’larını görebilir, geleceği göremez (mask ile).

* Cross-Attention → Encoder çıkışını kullanarak decoder token’larını context ile ilişkilendirir.

* FeedForward (FFN) → Token başına dönüşüm ve özellik zenginleştirme.

Ek olarak, LayerNorm, DropPath ve gamma parametreleri stabilite ve residual katkısı için kullanılır.

### 2️⃣ Gamma Parametreleri
```python
self.gamma_1 = nn.Parameter(torch.ones(embed_dim) * 1e-2)
self.gamma_2 = nn.Parameter(torch.ones(embed_dim) * 1e-2)
self.gamma_3 = nn.Parameter(torch.ones(embed_dim) * 1e-2)
```


Her gamma, residual bağlantının katkısını ölçekler.

* gamma_1 → Self-Attention çıktısı

* gamma_2 → Cross-Attention çıktısı

* gamma_3 → FeedForward çıktısı

> Küçük değerler (1e-2) → başlangıçta residual katkısı düşük → stabil öğrenme.

> Büyük değerler → daha hızlı katkı, risk: instabilite.

### 3️⃣ Forward Fonksiyon Adım Adım

```python 
def forward(self, x, enc_out=None, self_mask=None, enc_mask=None):
    # 1️⃣ Masked Self-Attention
    attn_out = self.self_attn(self.norm1(x), self.norm1(x), self.norm1(x), self_mask)
    x = x + self.drop_path(self.gamma_1 * attn_out)

    # 2️⃣ Cross-Attention
    if enc_out is not None:
        cross_out = self.cross_attn(self.norm2(x), self.norm2(enc_out), self.norm2(enc_out), enc_mask)
        x = x + self.drop_path(self.gamma_2 * cross_out)

    # 3️⃣ FeedForward
    ffn_out = self.ffn(self.norm3(x))
    x = x + self.drop_path(self.gamma_3 * ffn_out)

    return x


| Adım | İşlem                                             | Amaç                                                           |
| ---- | ------------------------------------------------- | -------------------------------------------------------------- |
| 1    | `self.norm1(x)` + `self.self_attn(...)`           | Decoder kendi geçmiş token’larıyla ilişki kurar (masked)       |
| 2    | `x + drop_path(gamma_1 * attn_out)`               | Residual bağlantı ve DropPath ile stabilizasyon                |
| 3    | `if enc_out is not None` → `self.cross_attn(...)` | Encoder’dan gelen bağlam bilgisi ile token’ları ilişkilendirir |
| 4    | `x + drop_path(gamma_2 * cross_out)`              | Cross-Attention residual eklenir                               |
| 5    | `self.norm3(x)` + `ffn(...)`                      | FeedForward ile token özellikleri zenginleştirilir             |
| 6    | `x + drop_path(gamma_3 * ffn_out)`                | FFN residual eklenir ve çıktı döner                            |


### 4️⃣ Self-Attention ve Cross-Attention Kullanımı

* Self-Attention her zaman çalışır, decoder kendi geçmiş token’larını görür.

* Cross-Attention sadece enc_out verilmişse çalışır:

```python
if enc_out is not None:
    cross_out = self.cross_attn(...)
```


* Yani modele hangi attention’ı kullanacağını enc_out parametresi ile aktarıyoruz.

* self_mask → self attention maskesi (geleceği görmeyi engeller)

* enc_mask → encoder maskesi (padding tokenlarını gizler)

### 5️⃣ Özet

* TransformerDecoderBlockLLM: Masked Self-Attention + Cross-Attention + FFN

* Gamma parametreleri → residual katkıyı ölçekler, stabilite sağlar

* Self ve Cross Attention aynı class içinde, cross_attention opsiyoneldir (enc_out varsa çalışır)

* DropPath → bloğu rastgele atlayarak regularization sağlar